## Bayesian Optimization with multiple objectives
Our pipeline should be as follows:
1. Define our design space as a grid or hyperplane etc
2. Define a model as surrogate to compute a score between target and a response query
3. Define acquistion function to score candidates
4. Define a selector to select candidate points
4. Define the optimization routine for the problem

In [1]:
import os
import torch
import time
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
import pdb

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

In [2]:
def branin(x):
    x1 = 15*x[0]-5
    x2 = 15*x[1]
    
    t1 = x2 - (x1**2)*(5.1/(4*np.pi**2)) + (5/np.pi)*x1 -6
    t2 = 10*(1-(1/(8*np.pi)))*np.cos(x1) + 10
    
    return t1**2 + t2

def currin(x):
    x1 = x[0]
    x2 = x[1]
    
    factor = (1-np.exp(-1/(2*x2)))
    numer = (2300*x1**3) + (1900*x1**2) + (2092*x1) + 60
    denom = 100*x1**3 + 500*x1**2 + 4*x1 +20
    
    return factor*(numer/denom)

def true_function(x):
    return -torch.stack([branin(x), currin(x)])

def batch_true_function(x):
    out = []
    for xi in x.squeeze(1):
        out.append(true_function(xi))
        
    return torch.stack(out, dim=0)

In [3]:
from botorch.test_functions.multi_objective import BraninCurrin
problem = lambda x : batch_true_function(x).to(**tkwargs)
ref_point = torch.tensor([-18,-6]).to(**tkwargs)

In [4]:
# 1. design space
from head import Grid
import numpy as np

X = np.linspace(0,1, num=100) 
Y = np.linspace(0,1, num=100)
grid = Grid(X,Y)

def generate_initial_data(n=6):
    points = torch.from_numpy(grid.points)
    soboleng = torch.quasirandom.SobolEngine(dimension=1)
    train_xid = torch.floor(soboleng.draw(n)*len(grid)).to(**tkwargs)
    train_x = points[train_xid.long(),:]
    train_obj = problem(train_x)
    
    return torch.squeeze(train_x), torch.squeeze(train_obj)

train_x, train_obj = generate_initial_data(n=6)


In [5]:
# 2. define your model
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    model = SingleTaskGP(train_x, train_obj)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    return mll, model

mll, model = initialize_model(train_x, train_obj)

In [6]:
# 3. Define acqusition function
from botorch.utils.multi_objective.box_decompositions.non_dominated import NondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler

partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_obj)
MC_SAMPLES = 128
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

acq_fun = lambda model: qExpectedHypervolumeImprovement(
    model=model,
    ref_point=ref_point.tolist(),  # use known reference point 
    partitioning=partitioning,
    sampler=sampler,
)

In [7]:
# 4. define a optimization routine for acqusition function 
from botorch.optim.optimize import optimize_acqf_discrete
from botorch.utils.transforms import unnormalize

BATCH_SIZE = 4 

def selector(f):
    choices = torch.from_numpy(grid.points)
    new_x, _ = optimize_acqf_discrete(
        acq_function = f,
        q=BATCH_SIZE,
        choices = choices
    )
    new_obj = problem(new_x)
    return new_x, new_obj


In [8]:
# 5. define the opitmization loop

from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume

torch.manual_seed(0)


N_ITERATIONS = 25

verbose = False
hv = Hypervolume(ref_point=ref_point)

hvs_all = []
hvs = []

# compute pareto front
pareto_mask = is_non_dominated(train_obj)
pareto_y = train_obj[pareto_mask]

# compute hypervolume
volume = hv.compute(pareto_y)
hvs.append(volume)

# run N_ITERATIONS rounds of BayesOpt after the initial random batch
for iteration in range(1, N_ITERATIONS + 1):    
    
    # fit the models
    fit_gpytorch_model(mll)

    # define the acquisition modules using a QMC sampler
    acquisition = acq_fun(model)

    # optimize acquisition functions and get new observations
    new_x, new_obj = selector(acquisition)

    # update training points
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])

    # compute pareto front
    pareto_mask = is_non_dominated(train_obj)
    pareto_y = train_obj[pareto_mask]
    
    # compute hypervolume
    volume = hv.compute(pareto_y)
    hvs.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    mll, model = initialize_model(train_x, train_obj)

    print(".", end="")

    hvs_all.append(hvs)


AttributeError: 'UpperConfidenceBound' object has no attribute 'X_pending'

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


iters = np.arange(N_ITERATIONS + 1) * BATCH_SIZE
max_hv = 59.36011874867746
log_hv_difference_qehvi = np.log10(max_hv - np.asarray(hvs_all))

fig, ax = plt.subplots()

ax.plot(
    iters, log_hv_difference_qehvi.mean(axis=0),
    label="qEHVI", linewidth=1.5,
)

ax.set(xlabel='number of observations (beyond initial points)', ylabel='Log Hypervolume Difference')
ax.legend(loc="lower right")

In [ ]:
from matplotlib.cm import ScalarMappable

fig, axes = plt.subplots(1, 1)
cm = plt.cm.get_cmap('viridis')

batch_number = torch.cat(
    [torch.zeros(6), torch.arange(1, N_ITERATIONS+1).repeat(BATCH_SIZE, 1).t().reshape(-1)]
).numpy()

sc = axes.scatter(train_obj[:, 0].cpu().numpy(), train_obj[:,1].cpu().numpy(), 
    c=batch_number, alpha=0.8,
)
axes.set_xlabel("Objective 1")
axes.set_xlim(-260, 5)
axes.set_ylim(-15, 0)
axes.set_ylabel("Objective 2")
norm = plt.Normalize(batch_number.min(), batch_number.max())
sm =  ScalarMappable(norm=norm, cmap=cm)
sm.set_array([])
cbar_ax = fig.add_axes([0.93, 0.15, 0.01, 0.7])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.ax.set_title("Iteration")